# Part 1: load and prepare data

In [136]:
# Include Libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
from torchmetrics.classification import MulticlassConfusionMatrix

from sklearn.model_selection import train_test_split

from pathlib import Path
import os
import shutil
import random

from PIL import Image
import numpy as np

from tqdm import tqdm

ModuleNotFoundError: No module named 'torchmetrics'

In [116]:
# Turn folders into Train and Test sets
for dirpath, dirnames, filenames in os.walk("data/asl_dataset"):
  for filename in (filenames):

    # generate random number between 1 and 10
    rand_int = random.randint(1, 10)

    # use rand_int to determine whether image goes to test or train

    if rand_int >= 9:
      split_path = Path("data/test")
    else:
      split_path = Path("data/train")

    # split the full path so we can pull the class folder name later
    sub_paths = dirpath.split("/")

    # generate souce path and destination path
    src_path = Path(dirpath) / filename
    dest_path = Path(split_path) / Path(sub_paths[len(sub_paths) - 1]) / filename

    # Copy image to location, or create destination then copy
    try:
      shutil.copyfile(src_path, dest_path)
    except IOError as io_err:
      os.makedirs(os.path.dirname(dest_path))
      shutil.copyfile(src_path, dest_path)

In [117]:
# check for proper test/train split (aproxamatly 20 %)
for dirpath, dirnames, filenames in os.walk(Path("data/asl_dataset")):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

print("\n")

for dirpath, dirnames, filenames in os.walk(Path("data/test")):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

print("\n")

for dirpath, dirnames, filenames in os.walk(Path("data/train")):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 36 directories and 0 images in 'data/asl_dataset'.
There are 0 directories and 70 images in 'data/asl_dataset/r'.
There are 0 directories and 70 images in 'data/asl_dataset/u'.
There are 0 directories and 70 images in 'data/asl_dataset/9'.
There are 0 directories and 70 images in 'data/asl_dataset/0'.
There are 0 directories and 70 images in 'data/asl_dataset/7'.
There are 0 directories and 70 images in 'data/asl_dataset/i'.
There are 0 directories and 70 images in 'data/asl_dataset/n'.
There are 0 directories and 70 images in 'data/asl_dataset/g'.
There are 0 directories and 70 images in 'data/asl_dataset/6'.
There are 0 directories and 70 images in 'data/asl_dataset/z'.
There are 0 directories and 70 images in 'data/asl_dataset/1'.
There are 0 directories and 70 images in 'data/asl_dataset/8'.
There are 0 directories and 65 images in 'data/asl_dataset/t'.
There are 0 directories and 70 images in 'data/asl_dataset/s'.
There are 0 directories and 70 images in 'data/asl_datase

In [118]:
# set up transformer
data_transform = transforms.Compose([
    # Resize our images to 64 x 64
    transforms.Resize(size=(64, 64)),
    #Turn image into torch.Tensor
    transforms.ToTensor()
])

In [119]:
# set up the directories to the images
train_dir = "data/train"
test_dir = "data/test"

# pull the images into datasets
train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,  # a transform for the data
                                  target_transform=None) # a transform for the label/target

test_data = datasets.ImageFolder(root=test_dir,
                                  transform=data_transform,  # a transform for the data
                                  target_transform=None) # a transform for the label/target

train_data, test_data  

(Dataset ImageFolder
     Number of datapoints: 2515
     Root location: data/train
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 2177
     Root location: data/test
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            ))

In [120]:
# set up dataloader
BATCH_SIZE = 8
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=False)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x15719ef50>,
 <torch.utils.data.dataloader.DataLoader at 0x157184950>)

In [121]:
class Net(nn.Module):
    def __init__(self, outputs):
        super(Net, self).__init__()
        c1out = 6
        c2out = 16

        self.conv1 = nn.Conv2d(3, c1out, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.norm1 = nn.BatchNorm2d(c1out)

        self.conv2 = nn.Conv2d(c1out, c2out, 3)
        self.norm2 = nn.BatchNorm2d(c2out)

        self.pooledOutputSize = c2out * 14 * 14 
        self.fc1 = nn.Linear( self.pooledOutputSize, 120 )
        self.fc2 = nn.Linear( 120, outputs )

    def forward(self, x):

        x = self.norm1(self.pool(F.relu(self.conv1(x))))

        x = self.norm2(self.pool(F.relu(self.conv2(x))))

        x = x.view(-1, self.num_flat_features(x)) 
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x
    
    #compute the output size after our convolution layers
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [122]:
def train( model, epochs, dataloader ): # One epoch uses the entire training set (one batch at a time) - 60,000 images in this case
    
    criterion = nn.CrossEntropyLoss() 
    optimizer = optim.Adam( model.parameters(), lr= 1e-4 ) 
    trainloader = dataloader

    for epoch in range( epochs ): # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate( trainloader, 0 ):
            # get the inputs; data is a list of [inputs, labels]

            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs) #predict the output with some training data
            loss = criterion(outputs, labels) #see how well we did

            loss.backward() #see how to change the weights to do better
            optimizer.step() #and actually change the weights

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    print('Finished Training')

In [123]:
net = Net(36)
train(net, 5, train_dataloader)

[1,   100] loss: 2.704
[1,   200] loss: 1.242
[1,   300] loss: 0.711
[2,   100] loss: 0.428
[2,   200] loss: 0.314
[2,   300] loss: 0.285
[3,   100] loss: 0.178
[3,   200] loss: 0.177
[3,   300] loss: 0.149
[4,   100] loss: 0.110
[4,   200] loss: 0.100
[4,   300] loss: 0.084
[5,   100] loss: 0.069
[5,   200] loss: 0.058
[5,   300] loss: 0.059
Finished Training


In [128]:
def evaluate( model, dataloader ):  

    testloader = dataloader
    correct = 0
    total = 0

    with torch.no_grad(): # <- Since we are not training, the model does not need to calculate gradients
        for data in testloader:
            images, labels = data
            outputs = model( images )
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Just do a coarse evaluation... how many did we predict correcly?
    print( 'Accuracy of the network on the 10000 test images: %f %%' % ( 100 * correct / total) )

In [129]:
evaluate(net, test_dataloader)

Accuracy of the network on the 10000 test images: 99.862196 %
